<a href="https://colab.research.google.com/github/mortgad/DLVR/blob/main/UTKface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# random bullshit go! 